<a href="https://colab.research.google.com/github/veronica1908/MARKETING_E2_A_III/blob/main/Preprocesamiento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
# Instalar la extensión ipython-sql
!pip install -q ipython-sql

# Cargar la extensión
%load_ext sql

# Instalar bibliotecas necesarias
!pip install sqlalchemy
!pip install mysql-connector-python

# Importar bibliotecas
import sqlite3
from sqlalchemy import create_engine
import pandas as pd


The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [16]:
# Conectarse a la base de datos
conn = sqlite3.connect('db_movies')
# Crear un cursor
cur = conn.cursor()

In [17]:
# Obtener información sobre las columnas de la tabla 'ratings'
cur.execute("PRAGMA table_info(ratings)")
columns_ratings = cur.fetchall()

# Imprimir el nombre de la columna y su tipo de datos para la tabla 'ratings'
print("Tipos de datos de las columnas en la tabla 'ratings':")
for column in columns_ratings:
    print(column[1] + ": " + column[2])

# Obtener información sobre las columnas de la tabla 'movies'
cur.execute("PRAGMA table_info(movies)")
columns_movies = cur.fetchall()

# Imprimir el nombre de la columna y su tipo de datos para la tabla 'movies'
print("\nTipos de datos de las columnas en la tabla 'movies':")
for column in columns_movies:
    print(column[1] + ": " + column[2])

Tipos de datos de las columnas en la tabla 'ratings':
userId: INTEGER
movieId: INTEGER
rating: REAL
timestamp: INTEGER

Tipos de datos de las columnas en la tabla 'movies':
movieId: INTEGER
title: TEXT
genres: TEXT


#Ajustes tabla movies

In [18]:
#Extraer el año de lanzamiento de la película en la tabla movies, el cual está incluido en el título de película
#Agregar una nueva columna para almacenar el año

conn = sqlite3.connect('db_movies')
cursor = conn.cursor()

# Verificar si la columna 'year' ya existe en la tabla 'movies'
cursor.execute("PRAGMA table_info(movies)")
columns = cursor.fetchall()
column_names = [col[1] for col in columns]
if 'year' not in column_names:
    # Si no existe, agregar la nueva columna
    cursor.execute("ALTER TABLE movies ADD COLUMN year INTEGER")

# Actualizar la nueva columna con el año extraído del título de la película
cursor.execute("""
    UPDATE movies
    SET year = CAST(SUBSTR(title, -5, 4) AS INTEGER)
""")

# Confirmar los cambios y cerrar la conexión
conn.commit()
conn.close()

In [19]:
conn = sqlite3.connect('db_movies')
cursor = conn.cursor()

# Verificar las columnas de la tabla 'movies'

# Obtener información sobre las columnas de la tabla 'movies'
cursor.execute("PRAGMA table_info(movies)")
columns_info = cursor.fetchall()

# Imprimir el nombre de la columna y su tipo de datos
for column in columns_info:
    print(column[1], "-", column[2])

# Cerrar la conexión a la base de datos
conn.close()

movieId - INTEGER
title - TEXT
genres - TEXT
year - INTEGER


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#Ajustes en la tabla ratings

In [ ]:
# verificamos el contenido de las primeras filas de la tabla ratings
conn = sqlite3.connect('db_movies')

query = "SELECT * FROM ratings LIMIT 5"

# Ejecutar la consulta y cargar los resultados en un DataFrame de Pandas
ratings = pd.read_sql(query, conn)

# Mostrar las primeras filas del DataFrame
print(ratings)
conn.close()

#Creación de nuevas tablas

In [ ]:
conn = sqlite3.connect('db_movies')

# Crear tabla con usuarios con más de 50 películas vistas
query = """
    DROP TABLE IF EXISTS usuarios_sel;
    CREATE TABLE usuarios_sel AS
    SELECT "userId" AS user_id, COUNT(*) AS cnt_rat
    FROM ratings
    GROUP BY "userId"
    HAVING cnt_rat > 50
    ORDER BY cnt_rat DESC;
"""

# Ejecutar la consulta
conn.executescript(query)

conn = sqlite3.connect('db_movies')
query = "SELECT * FROM usuarios_sel LIMIT 5"
# Ejecutar la consulta y cargar los resultados en un DataFrame de Pandas
usuarios_sel = pd.read_sql(query, conn)

# Mostrar las primeras filas del DataFrame
print(usuarios_sel)

conn.close()

In [ ]:
# Crear tabla con películas que han sido calificadas por más de 50 usuarios
conn = sqlite3.connect('db_movies')

query = """
DROP TABLE IF EXISTS movies_sel;

CREATE TABLE movies_sel AS
SELECT "movieId",
       COUNT(*) AS cnt_rat
FROM ratings
GROUP BY "movieId"
HAVING cnt_rat > 50
ORDER BY cnt_rat DESC;
"""

# Ejecutar la consulta
conn.executescript(query)
conn.close()

conn = sqlite3.connect('db_movies')
query = "SELECT * FROM movies_sel LIMIT 5"
# Ejecutar la consulta y cargar los resultados en un DataFrame de Pandas
movies_sel = pd.read_sql(query, conn)

# Mostrar las primeras filas del DataFrame
print(movies_sel)

conn.close()

In [ ]:
# Crear tablas filtradas de calificaciones con usuarios y películas
##ratings_final
conn = sqlite3.connect('db_movies')

query = """
DROP TABLE IF EXISTS ratings_final;

CREATE TABLE ratings_final AS
SELECT a."userId" AS user_id,
       a."movieId" AS movie_id,
       a."rating" AS rating
FROM ratings a
INNER JOIN movies_sel b ON a.movieId = b.movieId
INNER JOIN usuarios_sel c ON a."userId" = c.user_id;
"""
conn.executescript(query)
conn.close()

conn = sqlite3.connect('db_movies')
query = "SELECT * FROM ratings_final LIMIT 5"
# Ejecutar la consulta y cargar los resultados en un DataFrame de Pandas
ratings_final = pd.read_sql(query, conn)

# Mostrar las primeras filas del DataFrame
print(ratings_final)
conn.close()

In [ ]:
##movies_final
# Establecer conexión a la base de datos
conn = sqlite3.connect('db_movies')

# Definir la consulta SQL para crear la tabla movies_final
query = """
DROP TABLE IF EXISTS movies_final;

CREATE TABLE movies_final AS
SELECT a.movieId AS movie_id,
       a.title AS movie_title,
       a.genres,
       a.year
FROM movies a
INNER JOIN movies_sel c ON a.movieId = c.movieId;
"""

# Ejecutar la consulta
conn.executescript(query)

# Cerrar la conexión a la base de datos
conn.close()

conn = sqlite3.connect('db_movies')
query = "SELECT * FROM movies_final LIMIT 5"
# Ejecutar la consulta y cargar los resultados en un DataFrame de Pandas
movies_final = pd.read_sql(query, conn)

# Mostrar las primeras filas del DataFrame
print(movies_final)
conn.close()

In [ ]:
# Consulta SQL para seleccionar las primeras filas de la tabla full_ratings
conn = sqlite3.connect('db_movies')

query = "SELECT * FROM full_ratings LIMIT 5"  # Cambia el número de filas según tus necesidades

# Ejecutar la consulta y cargar los resultados en un DataFrame de Pandas
full_ratings = pd.read_sql(query, conn)

# Mostrar las primeras filas del DataFrame
print(full_ratings)

# Exportar la tabla preprocesada en SQL

In [ ]:
# Consulta SQL para seleccionar todos los datos de la tabla full_ratings
query = "SELECT * FROM full_ratings"

# Leer los datos en un DataFrame de pandas
df = pd.read_sql_query(query, conn)

# Guardar el DataFrame como un archivo CSV
df.to_csv('full_ratings.csv', index=False)

print("La tabla 'full_ratings' se ha exportado como full_ratings.csv")

# Cerrar la conexión
conn.close()

In [ ]:
#Verificar tablas creadas en el script
# Conectarse a la base de datos
conn = sqlite3.connect('db_movies')

# Crear un cursor
cur = conn.cursor()

# Obtener una lista de tablas en la base de datos
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cur.fetchall()

# Imprimir los nombres de las tablas para verificar la información
for table in tables:
    print(table[0])